<a href="https://colab.research.google.com/github/siyil/My-code/blob/master/firmware-diff-in-diff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
#from scipy.interpolate import spline
from scipy import stats
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols, quantreg
from ds_tools import db
from ds_tools import df_to_snowflake
import datetime as dt
from statsmodels.distributions.empirical_distribution import ECDF
from patsy import dmatrices

In [0]:

getmetric = '''
select * from dm.firmware_upgrade_analysis_Lishui_daily_off_minutes_stat
'''
runregression1 = """
select * from dm.firmware_upgrade_analysis_Lishui_daily_off_minutes
"""
runregression2 = """
select * from dm.firmware_upgrade_analysis_Lishui_daily_off_minutes_AA
"""

samples = db.query_snowflake(getmetric,warehouse = 'COMPUTE_WH')
mydata1 = db.query_snowflake(runregression1,warehouse = 'COMPUTE_WH')
mydata2 = db.query_snowflake(runregression2,warehouse = 'COMPUTE_WH')

samples = samples.assign(pval = lambda x: 1-stats.t.cdf(np.abs(x.tstat), x.control_n + x.test_n - 2))
samples = samples.assign(pvala = lambda x: 1-stats.t.cdf(np.abs(x.tstata), x.controla_n + x.testa_n - 2))
#samples1 = samples.assign(alpha = 1)
#samples5 = samples.assign(alpha = 5)
#samples10 = samples.assign(alpha = 10)
#samples = samples1.append(samples5).append(samples10)

#samples = samples.assign(upper = lambda x: x.diff_mean.astype(float) + stats.t.ppf(1- x.alpha/200, x.control_n + x.test_n - 2)*x.sp*np.sqrt(1/x.control_n + 1/x.test_n))
#samples = samples.assign(lower = lambda x: x.diff_mean.astype(float) - stats.t.ppf(1- x.alpha/200, x.control_n + x.test_n - 2)*x.sp*np.sqrt(1/x.control_n + 1/x.test_n))


In [118]:
mydata1 = mydata1.assign(after = 1.0)
mydata2 = mydata2.assign(after = 0.0)
mydata = mydata1.append(mydata2)
mydata = mydata.assign(test = (mydata['group_name'] == 'test')*1.0)
mydata = mydata.assign(inter = lambda x: x.test*x.after)
mydata['daily_off_minutes'] = mydata['daily_off_minutes'].astype('float')


experiments = list(mydata['experiment_name'].unique())
n_result = len(experiments)+1
regresults = {'experiment_name' : experiments + ['all'], 'DD':[0]*n_result, 'pvaldd':[0]*n_result}
reg = pd.DataFrame(regresults)


for exp in experiments:
  y, X = dmatrices('daily_off_minutes ~ after + test + inter', data=mydata.loc[mydata['experiment_name'] == exp], return_type='dataframe')
  model = sm.OLS(y,X).fit()
  reg.loc[reg['experiment_name'] == exp, ['DD','pvaldd']] = [model.params[3], model.pvalues[3]]
  
  
y, X = dmatrices('daily_off_minutes ~ after + test + inter', data=mydata, return_type='dataframe')
model = sm.OLS(y,X).fit()
reg.loc[reg['experiment_name'] == 'all', ['DD','pvaldd']] = [model.params[3], model.pvalues[3]]
print(reg)


                                     experiment_name           DD    pvaldd
0   Frankfurt_Lishui_S_3.0GE_Z_E_0.2_0.1_0.1_0.1_0.2   274.370589  0.000011
1      Munich_Lishui_S_3.0GE_Z_E_0.2_0.1_0.1_0.1_0.2    -0.071429  0.999448
2  Firmware_Experiment_Calgary_Lishui_S_V3.0_Z_U_...   -88.163866  0.322704
3  Firmware_Experiment_Cologne_Lishui_S_3.0GE_Z_E...   105.014286  0.865216
4  Firmware_Experiment_Berlin_Lishui_S_3.0GE_Z_E_...  1601.943482  0.000005
5  Firmware_Experiment_Paris_Lishui_S_V3.0_Z_E_0....  -276.981998  0.343136
6  Firmware_Experiment_Zurich_Lishui_S_V3.0_Z_E_0...   -14.202178  0.828993
7  Firmware_Experiment_Edmonton_Lishui_S_V3.0_Z_U...   128.268908  0.533437
8                                                all     5.040542  0.985456


In [109]:
print(reg)
samples = samples.merge(reg, on = 'experiment_name')
print(samples)

                                     experiment_name         DD    pvaldd
0   Frankfurt_Lishui_S_3.0GE_Z_E_0.2_0.1_0.1_0.1_0.2   4.653000  0.518195
1      Munich_Lishui_S_3.0GE_Z_E_0.2_0.1_0.1_0.1_0.2 -11.785143  0.200108
2  Firmware_Experiment_Calgary_Lishui_S_V3.0_Z_U_...   3.345276  0.667895
3  Firmware_Experiment_Cologne_Lishui_S_3.0GE_Z_E...  35.262726  0.416382
4  Firmware_Experiment_Berlin_Lishui_S_3.0GE_Z_E_...  40.729268  0.168337
5  Firmware_Experiment_Paris_Lishui_S_V3.0_Z_E_0.... -82.007711  0.074041
6  Firmware_Experiment_Zurich_Lishui_S_V3.0_Z_E_0... -10.688435  0.355782
7  Firmware_Experiment_Edmonton_Lishui_S_V3.0_Z_U...   2.956902  0.902976
8                                                all   9.599130  0.420544
                                     experiment_name         test_mean  \
0  Firmware_Experiment_Edmonton_Lishui_S_V3.0_Z_U...  233.974117711429   
1  Firmware_Experiment_Paris_Lishui_S_V3.0_Z_E_0....  392.689027501433   
2  Firmware_Experiment_Zurich_Lishui_S

In [0]:
samples.to_csv('/Users/siyi.luo/Code/IoT Experiment/stats.csv')